In [3]:
import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt

f = '/data/jlai/iris-hep/material-map/geant4_material_tracks.root'
file = uproot.open(f)
file['material-tracks'].keys()

['event_id',
 'v_x',
 'v_y',
 'v_z',
 'v_px',
 'v_py',
 'v_pz',
 'v_phi',
 'v_eta',
 't_X0',
 't_L0',
 'mat_x',
 'mat_y',
 'mat_z',
 'mat_r',
 'mat_dx',
 'mat_dy',
 'mat_dz',
 'mat_step_length',
 'mat_X0',
 'mat_L0',
 'mat_A',
 'mat_Z',
 'mat_rho',
 'mat_sx',
 'mat_sy',
 'mat_sz',
 'mat_ex',
 'mat_ey',
 'mat_ez']

In [10]:
f = '/data/jlai/iris-hep/OutputPT/output_pt_10/measurements.root'
file = uproot.open(f)
file['measurements'].keys()

['event_nr',
 'volume_id',
 'layer_id',
 'surface_id',
 'extra_id',
 'rec_loc0',
 'rec_loc1',
 'rec_time',
 'var_loc0',
 'var_loc1',
 'var_time',
 'clus_size',
 'channel_value',
 'channel_loc0',
 'clus_size_loc0',
 'channel_loc1',
 'clus_size_loc1',
 'true_loc0',
 'true_loc1',
 'true_phi',
 'true_theta',
 'true_qop',
 'true_time',
 'true_x',
 'true_y',
 'true_z',
 'true_incident_phi',
 'true_incident_theta',
 'residual_loc0',
 'residual_loc1',
 'residual_time',
 'pull_loc0',
 'pull_loc1',
 'pull_time']

In [ ]:
import uproot
import awkward as ak
import numpy as np

f = uproot.open('/data/jlai/iris-hep/material-map/geant4_material_tracks.root')
t = f["material-tracks"] 

branches = [
    "t_X0", "t_L0",     
    "v_eta",               
    "mat_step_length",     
    "mat_X0", "mat_L0", 
    "mat_rho"             
]
arr = t.arrays(branches, library="ak")

t_path        = ak.sum(arr["mat_step_length"], axis=1)
t_over_X0_sum = ak.sum(arr["mat_step_length"] / arr["mat_X0"], axis=1)
t_over_L0_sum = ak.sum(arr["mat_step_length"] / arr["mat_L0"], axis=1)
areal_density = ak.sum(arr["mat_rho"] * arr["mat_step_length"], axis=1)  # unit conversion may be needed

print("RMS( (Σ dℓ/X0) - t_X0 ) =", np.sqrt(np.mean((ak.to_numpy(t_over_X0_sum - arr["t_X0"]))**2)))
print("RMS( (Σ dℓ/L0) - t_L0 ) =", np.sqrt(np.mean((ak.to_numpy(t_over_L0_sum - arr["t_L0"]))**2)))

eta = ak.to_numpy(arr["v_eta"])
tx0 = ak.to_numpy(t_over_X0_sum)
# bin/plot as you like


RMS( (Σ dℓ/X0) - t_X0 ) = 0.0
RMS( (Σ dℓ/L0) - t_L0 ) = 0.0


In [29]:
# binning & profile
bins = np.linspace(-4.0, 4.0, 33)
idx = np.digitize(eta, bins) - 1
centers = 0.5 * (bins[1:] + bins[:-1])

means = np.array([tx0[idx==b].mean() if np.any(idx==b) else np.nan for b in range(len(bins)-1)])
errs  = np.array([tx0[idx==b].std(ddof=1)/np.sqrt((idx==b).sum()) if (idx==b).sum()>1 else np.nan
                  for b in range(len(bins)-1)])

pd.DataFrame({"eta_center": centers, "tx0_mean": means, "tx0_sem": errs})

,eta_center,tx0_mean,tx0_sem
0,-3.875,0.174683,0.000927
1,-3.625,0.189674,0.001005
2,-3.375,13.816240,0.774983
3,-3.125,69.261650,1.120447
4,-2.875,109.636879,0.048506
5,-2.625,109.761253,0.029149
6,-2.375,110.299477,0.015519
7,-2.125,111.674698,0.036716
8,-1.875,113.684471,0.049719
9,-1.625,122.953400,0.427237


In [ ]:
import uproot, numpy as np, pandas as pd

t = file["measurements"]

cols = ["volume_id","layer_id","true_x","true_y","true_z",
        "var_loc0","var_loc1","residual_loc0","residual_loc1","pull_loc0","pull_loc1"]
a = t.arrays(cols, library="pd")

a["R"] = np.sqrt(a.true_x**2 + a.true_y**2)
a["absZ"] = a.true_z.abs()

a["sigma_loc0"] = np.sqrt(a.var_loc0.clip(lower=0))
a["sigma_loc1"] = np.sqrt(a.var_loc1.clip(lower=0))

g = a.groupby(["volume_id","layer_id"])

layer_table = pd.DataFrame({
    "R_mean": g["R"].median(),
    "Z_abs_mean": g["absZ"].median(),
    "sigma_loc0_med": g["sigma_loc0"].median(),
    "sigma_loc1_med": g["sigma_loc1"].median(),
    "resid0_RMS": g["residual_loc0"].std(ddof=0),
    "resid1_RMS": g["residual_loc1"].std(ddof=0),
    "pull0_RMS": g["pull_loc0"].std(ddof=0),
    "pull1_RMS": g["pull_loc1"].std(ddof=0),
    "n_hits": g.size()
}).reset_index()

# Quick quality checks
# - pulls should have RMS ~ 1 per layer if variances are well calibrated
# - resid RMS should be ~ sigma if projection is simple & unbiased


In [13]:
layer_table 

,volume_id,layer_id,R_mean,Z_abs_mean,sigma_loc0_med,sigma_loc1_med,resid0_RMS,resid1_RMS,pull0_RMS,pull1_RMS,n_hits
0,17,2,32.212589,37.247871,0.015,0.015,0.014785,0.014759,0.985699,0.983948,128301
1,17,4,68.192879,37.338409,0.015,0.015,0.014800,0.014756,0.986673,0.983746,123291
2,17,6,114.205940,37.277042,0.015,0.015,0.014776,0.014788,0.985075,0.985848,119738
3,17,8,170.203369,37.248318,0.015,0.015,0.014798,0.014841,0.986519,0.989393,120775
4,24,2,259.958557,37.040970,0.043,1.200,0.042593,1.181592,0.990533,0.984660,115326
5,24,4,359.965088,36.994240,0.043,1.200,0.042334,1.175907,0.984511,0.979923,116720
6,24,6,499.968170,37.046844,0.043,1.200,0.042503,1.176368,0.988441,0.980307,117055
7,24,8,659.980103,37.037098,0.043,1.200,0.042306,1.177817,0.983861,0.981514,116083
8,29,2,820.219971,37.001289,0.072,NaN,0.070814,NaN,0.983521,NaN,219578
9,29,4,1020.168030,37.076996,0.072,NaN,0.070996,NaN,0.986050,NaN,235259


In [14]:
len(layer_table)

10

In [31]:
with open("/data/jlai/iris-hep-log/TrackingResolution-3.0/TrackingResolution-3.0/myODD.txt", "w") as f:
    f.write("#beam pipe\n")
    values = [0.00227,9999,9999,0.024]
    f.write(" ".join(str(v) for v in values) + "\n")

    f.write("#pixel\n")
    mean = 0.04879275
    for i in range(len(layer_table)):
        if i == 4:
            mean = 0.07509971
            f.write("#short strip\n")
        if i == 8:
            mean = 0.05986861
            f.write("#long strip\n")
        row = layer_table.iloc[i]
        position = row["R_mean"] * 0.001 # from cm to m
        sigma0 = row["sigma_loc0_med"] * 0.001 # from mm to m
        sigma1 = row["sigma_loc1_med"] * 0.001 # from mm to m
        
        values = [mean, sigma0, sigma1, position]
        f.write(" ".join(str(v) for v in values) + "\n")

In [4]:
file['material-tracks']['mat_r'].array(library='ak')[0], file['material-tracks']['mat_X0'].array(library='ak')[0], file['material-tracks']['mat_step_length'].array(library='ak')[0]

(<Array [23.6, 32.6, 32.8, 33.1, ..., 541, 541, 544, 546] type='432 * float32'>,
 <Array [353, 93.7, 284, 1.72e+03, ..., 17.7, 28.2, 425] type='432 * float32'>,
 <Array [8, 1.28, 2.56, 21.5, ..., 3.02, 30.2, 16.1, 3.02] type='432 * float32'>)

In [3]:
file['material-tracks']['mat_step_length'].array(library='ak')

<Array [[8, 1.28, 2.56, ..., 16.1, 3.02], ...] type='10000 * var * float32'>

In [2]:
file['material-tracks']['mat_X0'].array(library='ak')

<Array [[353, 93.7, 284, ..., 28.2, 425], ...] type='10000 * var * float32'>